In [44]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import bisect, brentq
from copy import copy

# Problem 1

In [2]:
class UpAndOutPut:

    def __init__(self, K, T, barrier, observationinterval):
        self.K = K
        self.T = T
        self.barrier = barrier
        self.observationinterval = observationinterval

In [3]:
hw1contract = UpAndOutPut(K=95, T=0.25, barrier=107, observationinterval=0.02)

In [4]:
class GBMdynamics:

    def __init__(self, S, r, rGrow, sigma=None):
        self.S = S
        self.r = r
        self.rGrow = rGrow
        self.sigma = sigma

    def update_sigma(self, sigma):
        self.sigma = sigma
        return self

In [5]:
hw1dynamics = GBMdynamics(S=100, sigma=0.4, rGrow=0, r=0)

In [ ]:
class TreeEngine:

    def __init__(self, N):
        self.N = N

    def price_upandout(self, dynamics, contract):

        deltat = contract.T / self.N
        J = np.ceil(np.log(contract.barrier/dynamics.S)/(dynamics.sigma*np.sqrt(3*deltat))-0.5)
        deltax = np.log(contract.barrier/dynamics.S)/(J+0.5)

        Sgrid = dynamics.S*np.exp(np.linspace(self.N, -self.N, num=2*self.N+1, endpoint=True)*deltax)
        #Here I decided to make the SMALLER indexes in this array correspond to HIGHER S

        numTimestepsPerObs = contract.observationinterval/deltat
        if abs(numTimestepsPerObs-round(numTimestepsPerObs)) > 1e-8:
            raise ValueError("This value of N fails to place the observation dates in the tree.")

        nu =        # complete this
        Pu =        # complete this
        Pd =        # complete this
        Pm =        # complete this

        optionprice = np.maximum(contract.K-Sgrid,0)   #an array of time-T option prices.

        #Next, induct backwards to time 0, updating the optionprice array
        #Hint: if x is an array, then what are x[2:] and x[1:-1] and x[:-2]

        for t in np.linspace(self.N-1, 0, num=self.N, endpoint=True)*deltat:
            # insert lines of code here if needed
            optionprice =  #complete this

        return optionprice[0]
        #The [0] is assuming that we are shrinking the optionprice array in each iteration of the loop,
        #until finally there is only 1 element in the array.
        #If instead you are keeping unchanged the size of the optionprice array in each iteration,
        #then you need to change the [0] to a different index.


In [ ]:
hw1tree=TreeEngine(N=100)

hw1tree.price_upandout(hw1dynamics, hw1contract)

# Problem 2

In [ ]:
# uses the same GBMdynamics class as in Problem 1

In [6]:
class CallOption:

    def __init__(self, K, T, price=None):
        self.K = K
        self.T = T
        self.price = price


In [48]:
class AnalyticEngine:

    def __init__(self):
        pass

    def BSpriceCall(self, dynamics, contract):
        # ignores contract.price if given, because this function calculates price based on the dynamics

        F = dynamics.S*np.exp(dynamics.rGrow*contract.T)
        std = dynamics.sigma*np.sqrt(contract.T)
        d1 = np.log(F/contract.K)/std+std/2
        d2 = d1-std
        return np.exp(-dynamics.r*contract.T)*(F*norm.cdf(d1)-contract.K*norm.cdf(d2))

    def IV(self, dynamics, contract):
        # ignores dynamics.sigma, because this function solves for sigma.

        if contract.price is None:
            raise ValueError('Contract price must be given')

        df = np.exp(-dynamics.r*contract.T)  #discount factor
        F = dynamics.S / df
        lowerbound = np.max([0,(F-contract.K)*df])
        C = contract.price
        if C<lowerbound:
            return np.nan
        if C==lowerbound:
            return 0
        if C>=F*df:
            return np.nan

        dytry = copy(dynamics)
        # We "try" values of sigma until we find sigma that generates price C

        # First find lower and upper bounds
        sigma_try = 0.2
        while self.BSpriceCall(dytry.update_sigma(sigma_try),contract)>C:
            sigma_try /= 2
        while self.BSpriceCall(dytry.update_sigma(sigma_try),contract)<C:
            sigma_try *= 2
        hi = sigma_try
        lo = hi/2
        # We have calculated "lo" and "hi" which bound the implied volatility from below and above.
        # In other words, the implied volatility is somewhere in the interval [lo,hi].
        # Then, to calculate the implied volatility within that interval,
        # for purposes of this homework, you may either (A) write your own bisection algorithm,
        # or (B) use scipy.optimize.bisect or (C) use scipy.optimize.brentq
        # You will need to provide lo and hi to those solvers.
        # There are other solvers that do not require you to bound the solution
        # from below and above (for instance, scipy.optimize.fsolve is a useful solver).
        # However, if you are able to bound the solution (of a single-variable problem),
        # then bisection or Brent will be more reliable.

        impliedVolatility =      # you fill this in, using bisect or brentq imported from scipy.optimize,
                                 # or by writing your own bisection algorithm.
                                 # See further instructions in last line of notebook which calls this function
        return impliedVolatility


In [ ]:
#Test the BSpriceCall function
hw1analytic = AnalyticEngine()
dynamics2 = GBMdynamics(sigma=0.4, rGrow=0, S=100, r=0)
contract2 = CallOption(K=100, T=0.5)
hw1analytic.BSpriceCall(dynamics2,contract2)

In [ ]:
#Test the IV function
contract2.price = 12
hw1analytic.IV(dynamics2,contract2)    # This code, EXACTLY AS WRITTEN HERE, must execute without crashing